Modelado Predictivo - Ventas Licorera "El Buen Gusto"

Estructura del Modelado:
1. **Preparación de datos**
   - Limpieza y tratamiento de valores faltantes
   - Creación de variables derivadas (features engineering)
-  - División en conjuntos de entrenamiento y prueba

2. **Desarrollo de modelos**
   - Implementar al menos 3 enfoques diferentes:
     - Modelo de series temporales (ARIMA, Prophet, etc.)
     - Modelo de machine learning (Random Forest, XGBoost, etc.)
     - Modelo de regresión múltiple
   - Validación cruzada temporal apropiada

3. **Evaluación y selección**
   - Métricas de evaluación: MAE, RMSE, MAPE
   - Comparación de modelos
   - Análisis de residuales


1. Configuración Inicial

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from sklearn.metrics import (mean_absolute_error, 
                            mean_squared_error, 
                            mean_absolute_percentage_error,
                            r2_score)
from sklearn.model_selection import RandomizedSearchCV
import joblib
import warnings
warnings.filterwarnings('ignore')